In [1]:
# IMPORTS
from ipywidgets import HBox, VBox, Label, Layout, AppLayout, Output
from IPython.display import display
import sys, os

# append parent of cwd to sys path to find modules
root_dir = os.path.join(os.getcwd(), '..')
sys.path.append(root_dir)

# surpress pygame version prompt
os.environ['PYGAME_HIDE_SUPPORT_PROMPT'] = "hide"

# enable matplotlib widgets
%matplotlib widget

from src.app import App
from src.datatypes.call_response_set import CallResponseSet
from dashboards.widgets import inputHeading, inputSelect, inputUpload, generationHeading, checkpointSelect, tempSlider, adaptationHeading, stepsSelect, startButton, applySettingsButton
from dashboards.widgets import h2_heading 

from src.io.plot import multitrack_pianoroll
from src.io.output import play_button
from src.io.input import loadMidiFile

C:\Users\Eric\miniconda3\envs\mpro\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [2]:
##########################
#C#    OUTPUT & APP    ###
##########################

log = Output()
output = Output()

pianoroll_container = Output()
play_button_container = Output()
meta_container = Output()

# init
app = App(log=log)

In [3]:
##########################
#M#   SELECTION DATA   ###
##########################

adaptation_operations = app.get_adaptation_operations()
generators = app.get_generators()
demo_melodies = app.get_demo_input()

checkpointSelect.options = [(g[0], generators.index(g)) for g in generators]
stepsSelect.options = adaptation_operations
inputSelect.options = [(d[0], demo_melodies.index(d)) for d in demo_melodies]

In [4]:
##########################
#C#     CONTROLLER     ###
##########################

def apply_settings(b):
    app.apply_settings(checkpointSelect.value, tempSlider.value, stepsSelect.value)
    pass

def run_adaptation(b):
    # TODO handle file upload
    input_path = demo_melodies[inputSelect.value][1]
    cr_set = app.run(input_path)

    # rest output containers
    pianoroll_container.clear_output()
    play_button_container.clear_output()

    # create pianoroll plots from results and print them
    midi_list = [cr_set.input_sequence.sequence, cr_set.generated_base_sequence.sequence, cr_set.output_sequence.sequence]
    name_list = ['Input', 'Generation', 'Result']
    multitrack_pianoroll(midi_list, name_list, pianoroll_container)

    # display play buttons for audio
    with play_button_container: 
        display(h2_heading('Input:'))
    play_button(midi_list[0], play_button_container)
    with play_button_container: 
        display(h2_heading('Generation:'))    
    play_button(midi_list[1], play_button_container)
    with play_button_container: 
        display(h2_heading('Adaptation:'))
    play_button(midi_list[2], play_button_container)

    # create evaluation plots

    # print meta data
    with meta_container:
        print('Input Analysis', cr_set.input_sequence.analysis)
        print('Gen Analysis', cr_set.output_sequence.analysis)
        print('Gen Meta', cr_set.generated_base_sequence.meta)
        print('Adaptation Meta', cr_set.output_sequence.meta)



startButton.on_click(run_adaptation)
applySettingsButton.on_click(apply_settings)

In [5]:
##########################
#V#  SETTINGS SIDEBAR  ###
##########################

side_bar_layout = Layout(border='1px solid grey', padding='10px')

settings_box = VBox([
        generationHeading,
        HBox([Label('Model & Checkpoint:'), checkpointSelect]),
        HBox([Label('Temperature:'), tempSlider]),
        adaptationHeading,
        HBox([Label('Steps:'), stepsSelect]),
        applySettingsButton,
        inputHeading,
        inputSelect,
        HBox([Label('Or upload own file:'), inputUpload]),
        startButton,
    ], layout = side_bar_layout)

In [6]:
##########################
#V#    RESULTS AREA    ###
##########################

main_layout = Layout(border='1px solid grey', padding='10px')

pianorolls = HBox([pianoroll_container, play_button_container])

results_box = VBox([pianorolls, meta_container, output], layout=main_layout)

In [7]:
##########################
#V#     APP LAYOUT     ###
##########################

AppLayout(left_sidebar=settings_box,
          center=results_box,
          header=None,
          footer=log,
          right_sidebar=None,
          pane_widths=[1,2,0],
          pane_heights=[0,4,1])

AppLayout(children=(Output(layout=Layout(grid_area='footer')), VBox(children=(HTML(value='<h2>Generation Setti…

In [9]:
run_adaptation(startButton)

midis [<pretty_midi.pretty_midi.PrettyMIDI object at 0x0000018130099708>, <pretty_midi.pretty_midi.PrettyMIDI object at 0x000001811D8EE488>, <pretty_midi.pretty_midi.PrettyMIDI object at 0x00000181300A0508>]
names ['Input', 'Generation', 'Result']
